# Roman Galactic Plane Survey Region Design

In 2024, the astronomical community submitted a set of White Papers and Science Pitches regarding the science that could be done with a Roman survey of the Galactic Plane.  Many of these contributions identified numerous specific regions of interest, and preferred filters for the observations.  

In this notebook, we extract the specified regions and filter sets with the goal of combining them into single overall desired survey footprint.  

In [ ]:
import config_utils
import survey_footprints
import regions
import healpy as hp
from mw_plot import MWSkyMap, MWSkyMapBokeh
from astropy_healpix import HEALPix
from astropy import units as u
from astropy.coordinates import Galactic, TETE, SkyCoord
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
from os import path
%matplotlib inline


# Configure path to local repository
root_dir = '/Users/rstreet/software/rgps'

File config/rgps_survey_regions.json contains the desired survey regions from each community contribution, described in machine-readable form.

Note that not all of the science cases specified explicit regions or filter selections.  Those that did have been marked 'ready_for_use': 'True'; further details will be sought from the authors for the remaining science cases. 

In [ ]:
config = config_utils.read_config(path.join(root_dir, 'config', 'rgps_science_cases.json'))

Regions of interest are specified on a per-filter basis, and replicated for multiple filters if this more than one bandpass was requested for the region.  This will be used later on to build combined survey maps for each filter.

Our next step is to extract a dictionary of the desired regions for each filter, grism and prism that Roman offers.

In [ ]:
optical_components = ['F087', 'F106', 'F129', 'F158', 'F184', 'F213', 'F146', 'G150', 'P127']
requested_regions = {optic: [] for optic in optical_components}

for author in config.keys():
    info = config[author]
    if info['ready_for_use']:
        for optic in optical_components:
            if optic in info.keys():
                for region in info[optic]:
                    region['label'] = author
                    region['optic'] = optic
                    requested_regions[optic].append(region)

requested_regions

Desired regions are defined as one of the following options:
a) a box in galactic longitude, latitude (l [l_min, l_max], b [b_min, b_max]) 
b) a single pointing, in which case the parameters given are l_center, b_center, field radius,   
c) a pre-defined HEALpixel map.  This option is most commonly used for synergies with other surveys. 

For the record, the WFI has a field of view of 0.281 sq.deg. and is approximated with a circle of radius 0.3 deg.

In the process of converting each region to a list of in-region HEALpixels, we also assign each pixel a value of 1.  This will be used effectively like a 'vote' for that pixel when the regions are combined later. 

In [ ]:
desired_regions = {}
regions_by_author = {}

for optic, region_list in requested_regions.items():
    regions_for_optic = []
    for box in region_list:
        if 'catalog' in box.keys():
            region_set = regions.create_region_set(box)
        else:
            region_set = [ regions.create_region(box) ]

        for r in region_set:
            if r.label in regions_by_author.keys():
                regions_by_author[r.label].append(r)
            else:
                regions_by_author[r.label] = [r]
            
            # If the region is valid, the list of included pixels will be non-zero. 
            # Each pixel within a region is given a value of 1 - essentially being a 'vote' for that pixel, 
            # for each science case.
            if len(r.pixels) > 0:
                r.pixel_priority = np.zeros(r.NPIX)
                r.pixel_priority[r.pixels] = 1.0
                r.predefined_pixels = True
                r.make_map()
                
                regions_for_optic.append(r)
        
    desired_regions[optic] = regions_for_optic

for optic, regions_for_optic in desired_regions.items():
    if len(regions_for_optic) > 0:
        print(optic + ' has region(s):')
        for r in regions_for_optic:
            print(r.summary())
    else:
        print(optic + ' has no requested regions')

Let's inspect an example region map, for illustration

In [ ]:
# Random example is the Kruszynska Science Pitch for synergies with Rubin because it's a complex region shape
# This returns a set of regions because this author asked for observations in multiple filters
rlist = regions_by_author['Kruszynska']
for r in rlist:
    print(r.summary())
    r.sky_plot()

## Building a combined survey region

With our set of desired survey regions now defined, we can combine them to form the overall survey region. 
Since all selected HEALpixels have a value of 1.0 and zero elsewhere, we can just co-add the region maps for each filter, to see which sky regions are highest priority.  

In [ ]:
# Dictionary of the combined survey regions per optical component
combined_regions = {}

# In order to use the plotting method of the CelestialRegion object, we can create separate regions for the combined maps 
for optic, region_list in desired_regions.items():
    if len(region_list) > 0:
        r_merge = regions.combine_regions(region_list)
        r_merge.optic = optic
        r_merge.label = 'Combined survey footprint'

        mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
        mw1.title = r_merge.label + ' ' + r_merge.optic
        s = r_merge.pixels_to_skycoords()
        mw1.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=5, alpha=0.4)
        plt.rcParams.update({'font.size': 22})

        plt.tight_layout()
        plt.savefig(path.join(root_dir, 'survey_maps', 'survey_map_'+r_merge.optic+'.png'))
                    
        combined_regions[optic] = r_merge

In the plots above, the color of each survey tile scales according to the number of science cases that requested that tile be observed in the given filter, with deep red indicating the highest number of 'votes' for a tile.  

We can also plot interactive versions of this plot, although unfortunately only the optical background image seems to be available.  

In [ ]:
mw2 = MWSkyMapBokeh()
s = combined_regions['F213'].pixels_to_skycoords()
mw2.scatter(s.ra.deg*u.deg, s.dec.deg*u.deg, c="r", s=5, alpha=0.4)
mw2.show()